Hyperparameter Tuning

In [1]:
import os
import time
from datetime import datetime

import optuna
import torch
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import (Callback, EarlyStopping,
                                         ModelCheckpoint)
from lightning.pytorch.loggers import TensorBoardLogger

from common.file_paths import BASE_DIR
from modules import (LightningTransformerMinecraftStructureGenerator,
                     MinecraftDataModule)


class MetricsTracker(Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_val_accuracy = 0

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss')
        val_accuracy = trainer.callback_metrics.get('val_accuracy')
        if val_loss is not None and val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
        if val_accuracy is not None and val_accuracy > self.best_val_accuracy:
            self.best_val_accuracy = val_accuracy


def get_inference_time(model, datamodule):
    # Switch to evaluation mode for inference
    model.eval()

    # Get a single sample from the validation set
    val_dataloader = datamodule.val_dataloader()[0]
    prompt = next(iter(val_dataloader.dataset))[1][0]

    # Measure inference time for a single sample
    torch.cuda.synchronize()  # Ensure CUDA synchronizes before starting timing
    start_time = time.time()
    with torch.no_grad():
        model.generate(prompt, autoregressive=True)
    torch.cuda.synchronize()  # Ensure CUDA synchronizes before stopping timing
    end_time = time.time()

    inference_time = end_time - start_time

    return inference_time


def objective(trial: optuna.Trial):
    seed_everything(0, workers=True)

    lightning_model = LightningTransformerMinecraftStructureGenerator(
        num_classes=20,
        max_sequence_length=512,
        embedding_dim=trial.suggest_categorical(
            "embedding_dim", [16, 32, 64, 128]),
        embedding_dropout=trial.suggest_float(
            "embedding_dropout", 0.1, 0.5, step=0.1),
        decoder_dim=trial.suggest_categorical(
            "decoder_dim", [32, 64, 128, 256]),
        num_heads=trial.suggest_categorical("num_heads", [4, 8, 16, 32]),
        num_layers=trial.suggest_int("num_layers", 2, 4),
        decoder_dropout=trial.suggest_float(
            "decoder_dropout", 0.1, 0.5, step=0.1),
        freeze_encoder=True,
        learning_rate=trial.suggest_float(
            "learning_rate", 1e-5, 1e-3, log=True)
    )

    hdf5_file = os.path.join(BASE_DIR, 'data.h5')
    data_module = MinecraftDataModule(
        file_path=hdf5_file,
        batch_size=trial.suggest_categorical("batch_size", [8, 16, 32]),
    )

    logger = TensorBoardLogger(
        'lightning_logs', name='minecraft_structure_generator', log_graph=False)
    metrics_tracker = MetricsTracker()
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        mode='min',
        save_last=True,
        save_weights_only=True
    )
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        patience=30,
        verbose=False,
        mode='min'
    )

    trainer = Trainer(
        max_epochs=5000,
        logger=logger,
        gradient_clip_val=1.0,
        log_every_n_steps=5,
        callbacks=[
            checkpoint_callback,
            early_stop_callback,
            metrics_tracker
        ]
    )

    start_time = time.time()

    trainer.fit(lightning_model, datamodule=data_module)

    end_time = time.time()
    training_time = end_time - start_time

    inference_time = get_inference_time(lightning_model, data_module)

    return metrics_tracker.best_val_loss, metrics_tracker.best_val_accuracy, training_time, inference_time


if __name__ == "__main__":
    # study_name = "study_20231204195755"
    study_name = f"study_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    study = optuna.create_study(directions=("minimize", "maximize", "minimize", "minimize"),
                                study_name=study_name, storage='sqlite:///studies.db', load_if_exists=True)
    study.optimize(objective, n_trials=100)

    print("Best hyperparameters:", study.best_trial.params)

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-12-05 17:08:41,401] A new study created in RDB with name: study_20231205170841
Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: lightning_logs\minecraft_structure_generator


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                                   | Params
-----------------------------------------------------------------
0 | model | TransformerMinecraftStructureGenerator | 66.8 M
-----------------------------------------------------------------
434 K     Trainable params
66.4 M    Non-trainable params
66.8 M    Total params
267.189   Total estimated model params size (MB)


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  6.78it/s]

c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


c:\Users\mmmfr\Documents\Repositories\minecraft-schematic-generator\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


...

Epoch 278: 100%|██████████| 19/19 [00:03<00:00,  5.51it/s, v_num=15]


[I 2023-12-05 22:26:48,740] Trial 15 finished with values: [0.010125442408025265, 0.9979034662246704, 1044.3304104804993, 3.500105381011963] and parameters: {'embedding_dim': 128, 'embedding_dropout': 0.0, 'decoder_dim': 64, 'num_heads': 16, 'num_layers': 2, 'decoder_dropout': 0.2, 'learning_rate': 0.00032314535304490776, 'batch_size': 32}. 
Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                                   | Params
-----------------------------------------------------------------
0 | model | TransformerMinecraftStructureGenerator | 66.8 M
-----------------------------------------------------------------
459 K     Trainable params
66.4 M    Non-trainable params
66.8 M    Total params
267.291   Total estimated model params size (MB)


Epoch 287: 100%|██████████| 9/9 [00:04<00:00,  1.99it/s, v_num=16]


[I 2023-12-05 22:51:59,829] Trial 16 finished with values: [0.19784489274024963, 0.9325544238090515, 1506.3476374149323, 4.002651691436768] and parameters: {'embedding_dim': 16, 'embedding_dropout': 0.0, 'decoder_dim': 32, 'num_heads': 16, 'num_layers': 3, 'decoder_dropout': 0.1, 'learning_rate': 0.008719601420374878, 'batch_size': 64}. 
Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                                   | Params
-----------------------------------------------------------------
0 | model | TransformerMinecraftStructureGenerator | 67.7 M
-----------------------------------------------------------------
1.3 M     Trainable params
66.4 M    Non-trainable params
67.7 M    Total params
270.764   Total estimated model params size (MB)


Epoch 104:  40%|████      | 31/77 [00:03<00:04, 10.22it/s, v_num=17]       

Running Best Model

In [2]:
import os

import optuna
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.profilers import SimpleProfiler

from common.file_paths import BASE_DIR
from modules import (GenerateSchematicCallback,
                     LightningTransformerMinecraftStructureGenerator,
                     MinecraftDataModule)

study_name = 'study_20231205030812'
storage_url = 'sqlite:///studies.db'

study = optuna.load_study(study_name=study_name, storage=storage_url)


def get_nth_best_trial(study, n, objective_id=0):
    # Sort the completed trials of the study object by the specified objective value.
    sorted_trials = sorted(
        study.trials, key=lambda t: t.values[objective_id] if t.values is not None else float('inf'))

    # Return the parameters of the n-th best trial.
    return sorted_trials[n].params


objective_id = 1
x = 0
nth_best_params = get_nth_best_trial(study, x, objective_id=0)
print(nth_best_params)

seed_everything(1, workers=True)

lightning_model = LightningTransformerMinecraftStructureGenerator(
    num_classes=20,
    max_sequence_length=512,
    freeze_encoder=True,
    **nth_best_params
)

hdf5_file = os.path.join(BASE_DIR, 'data.h5')
data_module = MinecraftDataModule(
    file_path=hdf5_file,
    batch_size=32,
    # num_workers=4
)

logger = TensorBoardLogger(
    'lightning_logs', name='minecraft_structure_generator', log_graph=False)
profiler = SimpleProfiler()
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_last=True
)
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=False,
    mode='min'
)
generate_schematic_callback = GenerateSchematicCallback(
    save_path='schematic_viewer/public/schematics/',
    data_module=data_module,
    generate_train=False,
    generate_val=True,
    generate_all_datasets=False,
    generate_every_n_epochs=10,
    autoregressive=True
)

trainer = Trainer(
    max_epochs=5000,
    logger=logger,
    profiler=profiler,
    gradient_clip_val=1.0,
    log_every_n_steps=5,
    callbacks=[
        checkpoint_callback,
        early_stop_callback,
        generate_schematic_callback
    ]
)

trainer.fit(lightning_model, datamodule=data_module)

Seed set to 1


{'embedding_dim': 64, 'embedding_dropout': 0.5, 'decoder_dim': 32, 'num_heads': 2, 'num_layers': 3, 'decoder_dropout': 0.2, 'learning_rate': 0.0006395506262517658}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                                   | Params
-----------------------------------------------------------------
0 | model | TransformerMinecraftStructureGenerator | 66.9 M
-----------------------------------------------------------------
489 K     Trainable params
66.4 M    Non-trainable params
66.9 M    Total params
267.410   Total estimated model params size (MB)


Epoch 408: 100%|██████████| 21/21 [00:03<00:00,  5.57it/s, v_num=22]


FIT Profiler Report

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                               	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                                	|  -     